# Perform Sentinel-1 InSAR using ESA SNAP Toolkit using SageMaker geospatial capabilities

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

---

## Introduction

In this notebook, we will demonstrate how to use the European Space Agency (ESA) [Sentinel Application Platform (SNAP) Toolkit](https://step.esa.int/main/toolboxes/snap/) from within Amazon SageMaker to perform InSAR following [this guide](http://step.esa.int/docs/tutorials/command_line_inSAR_processing.pdf) from ESA. This notebook will generate an interferogram that visualizes the surface deformation caused by the April 16th, 2016 Kumamoto earthquake in Japan.

#### Why use Sentinel-1?
As Sentinel-1 carries a Synthetic Aperature Radar (SAR) instrument, which has advantages over optical sensors in that it can see through clouds and weather, and can be used at night.

#### Making changes
If you want to make changes to the processing steps without having to download the scenes and unzip them again (as these steps can take awhile), simply delete the "out" folder within data/snap/kumamoto, restart the Kernel and clear all outputs, make your changes, and then run the notebook!

## Prerequisites

This notebook runs with the Geospatial 1.0 kernel with a `ml.geospatial.interactive` instance. Note that the following policies need to be attached to the execution role that you used to run this notebook:
- AmazonSageMakerFullAccess
- AmazonSageMakerGeospatialFullAccess

You can see the policies attached to the role in the IAM console under the permissions tab. If required, add the roles using the 'Add Permissions' button.

In addition to these policies, ensure that the execution role's trust policy allows the SageMaker-GeoSpatial service to assume the role. This can be done by adding the following trust policy using the 'Trust relationships' tab:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "sagemaker.amazonaws.com",
                    "sagemaker-geospatial.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

## Get temporary S3 credentials for NASA EOSDIS
SageMaker Geospatial currently does not have Sentinel-1 SLCs which are what we need for interferometry, so we will fetch the needed SLCs from NASA directly.

We will want to download 2 scenes, before the earthquake and after, so we can "see" the phase shift or changes in the earth's surface due to the earthquake.  To get files from NASA on AWS, you will need to get a set of temporary S3 credentials which you can fetch from NASA EarthSearch.

To access the data, you need to have a login on NASA EarthSearch portal, you can register and login with the link below. The data are available to all and the login is free of charge. 
- https://urs.earthdata.nasa.gov/

After you logged in, you can obtain the necessary S3 credentials by navigating to the following page:
- https://sentinel1.asf.alaska.edu/s3credentials

Copy the provided credentials into the cell below.

In [ ]:
# set temporary AWS Credentials - NOTE that these creds expire every hour
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_SESSION_TOKEN = ""
AWS_REGION = "us-west-2"  # do not change the region

### Get before and after Sentinel-1 scenes

In [ ]:
import boto3
import os

# set up our connection to the NASA S3 bucket
session = boto3.Session()
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
)
bucket = "asf-ngap2w-p-s1-slc-7b420b89"

# create a local folder to store Kumamoto data
if not os.path.exists(os.getcwd() + "/data/snap/kumamoto"):
    print("Creating: " + os.getcwd() + "/data/snap/kumamoto")
    os.makedirs(os.getcwd() + "/data/snap/kumamoto")

In [ ]:
# Before Scene: S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB
# s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip
obj_name = "S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip"
trim_filename = obj_name[: len(obj_name) - 4]
before_folder_name = "data/snap/kumamoto/" + trim_filename + ".SAFE"
file_name = "data/snap/kumamoto/" + obj_name
print(before_folder_name)
print(file_name)

In [ ]:
# Before Scene: create directory to put the file and expand it
os.environ["SENTINEL1_BEFORE_TILE_PATH"] = before_folder_name
os.environ["SENTINEL1_BEFORE_TILE_ID"] = trim_filename

!echo $SENTINEL1_BEFORE_TILE_PATH
!echo $SENTINEL1_BEFORE_TILE_ID

In [ ]:
# Before Scene: download the file if we don't already have it
if not os.path.exists(file_name):
    s3.download_file(bucket, obj_name, file_name)
    print(file_name + " downloaded")

In [ ]:
# Before Scene: expand the zip if we haven't already
if not os.path.exists(before_folder_name):
    import zipfile

    with zipfile.ZipFile(file_name, "r") as zip_ref:
        zip_ref.extractall("data/snap/kumamoto/")

In [ ]:
# After Scene: S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE
# s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE.zip
obj_name = "S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE.zip"
trim_filename = obj_name[: len(obj_name) - 4]
after_folder_name = "data/snap/kumamoto/" + trim_filename + ".SAFE"
file_name = "data/snap/kumamoto/" + obj_name
print(after_folder_name)
print(file_name)

In [ ]:
# After Scene: create directory to put the file and expand it
os.environ["SENTINEL1_AFTER_TILE_PATH"] = after_folder_name
os.environ["SENTINEL1_AFTER_TILE_ID"] = trim_filename

!echo $SENTINEL1_AFTER_TILE_PATH
!echo $SENTINEL1_AFTER_TILE_ID

sentinel1_after_tile_id = trim_filename

In [ ]:
# After Scene: download the file if we don't already have it
if not os.path.exists(file_name):
    s3.download_file(bucket, obj_name, file_name)
    print(file_name + " downloaded")

In [ ]:
# After Scene: expand the zip if we haven't already
if not os.path.exists(after_folder_name):
    import zipfile

    with zipfile.ZipFile(file_name, "r") as zip_ref:
        zip_ref.extractall("data/snap/kumamoto/")

## Setup SageMaker geospatial capabilities and imports

In [ ]:
import sagemaker
import json

execution_role = sagemaker.get_execution_role()
geospatial_client = session.client(service_name="sagemaker-geospatial")

## Process SAR data with SNAP gpt

In [ ]:
# uncomment to see how to use this gpt operator
#!gpt TOPSAR-Split -h

In [ ]:
# Before Scene: we must split before we can coregister
!gpt TOPSAR-Split -t data/snap/kumamoto/out/${SENTINEL1_BEFORE_TILE_ID}_Split_Stack.dim \
    ${SENTINEL1_BEFORE_TILE_PATH}/manifest.safe \
    -Psubswath=IW1 \
    -PselectedPolarisations=VV

In [ ]:
# After Scene: we must split before we can coregister
!gpt TOPSAR-Split -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Split_Stack.dim \
    ${SENTINEL1_AFTER_TILE_PATH}/manifest.safe \
    -Psubswath=IW1 \
    -PselectedPolarisations=VV

In [ ]:
## Apply Orbit file to each
# if you need a precise orbit file, get it from here: https://scihub.copernicus.eu/gnss/#/home
!gpt Apply-Orbit-File -t data/snap/kumamoto/out/${SENTINEL1_BEFORE_TILE_ID}_Orb.dim \
    data/snap/kumamoto/out/${SENTINEL1_BEFORE_TILE_ID}_Split_Stack.dim \
    -PorbitType="Sentinel Precise (Auto Download)" \
    -PpolyDegree=3

In [ ]:
!gpt Apply-Orbit-File -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Orb.dim \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Split_Stack.dim  \
    -PorbitType="Sentinel Precise (Auto Download)" \
    -PpolyDegree=3

In [ ]:
## Coregister the stack
!gpt Back-Geocoding -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Coreg_Stack.dim \
    data/snap/kumamoto/out/${SENTINEL1_BEFORE_TILE_ID}_Orb.dim \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Orb.dim \
    -PdemName="SRTM 3Sec" \
    -PdemResamplingMethod=BILINEAR_INTERPOLATION \
    -PexternalDEMNoDataValue=0.0 \
    -PresamplingType=BILINEAR_INTERPOLATION \
    -PmaskOutAreaWithoutElevation=true \
    -PoutputRangeAzimuthOffset=false \
    -PoutputDerampDemodPhase=false \
    -PdisableReramp=false

In [ ]:
## Create Interferogram using coregistered data
!gpt Interferogram -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Ifg.dim  \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Coreg_Stack.dim \
    -PsubtractFlatEarthPhase=true \
    -PsrpPolynomialDegree=5 \
    -PsrpNumberPoints=501 \
    -PorbitDegree=3 \
    -PincludeCoherence=true \
    -PcohWinAz=2 \
    -PcohWinRg=10 \
    -PsquarePixel=true \
    -PsubtractTopographicPhase=false \
    -PdemName="SRTM 3Sec" \
    -PexternalDEMNoDataValue=0.0 \
    -PtileExtensionPercent=100 \
    -PoutputElevation=false \
    -PoutputLatLon=false

In [ ]:
## TOPS Deburst 
!gpt TOPSAR-Deburst -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Deburst.dim  \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Ifg.dim \
    -PselectedPolarisations=VV

In [ ]:
## TopoPhaseRemoval
!gpt TopoPhaseRemoval -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Topo.dim \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Deburst.dim \
    -PdemName="SRTM 3Sec" \
    -PexternalDEMNoDataValue=0.0 \
    -PtileExtensionPercent=100 \
    -PoutputTopoPhaseBand=false \
    -PoutputElevationBand=false \
    -PoutputLatLonBands=false

In [ ]:
## Multilook
!gpt Multilook -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_ML.dim \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Topo.dim \
    -PnAzLooks=2 \
    -PnRgLooks=6 \
    -PoutputIntensity=false \
    -PgrSquarePixel=false

In [ ]:
## GoldsteinPhaseFiltering
!gpt GoldsteinPhaseFiltering -t data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_Gold.dim \
    data/snap/kumamoto/out/${SENTINEL1_AFTER_TILE_ID}_ML.dim \
    -Palpha=1.0 \
    -PFFTSizeString=64 \
    -PwindowSizeString=3 \
    -PuseCoherenceMask=false \
    -PcoherenceThreshold=0.2

## Visualize output interferogram

Next we'll visualize the processed data. In order to read the data in BEAM-DIMAP format into an xarray structure, we'll use the `nd` library. This library can be installed by executing the cell below.

In [ ]:
%pip install nd

In [ ]:
# visualize
import nd

finalDIM = nd.io.open_beam_dimap(f"data/snap/kumamoto/out/{sentinel1_after_tile_id}_Gold.dim")

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12))
ax = plt.axes()
finalDIM["i_ifg_VV_08Apr2016_20Apr2016"].plot(ax=ax, robust=True)
plt.show()

In [ ]:
# clip and zoom in so we can see the phase fringes
from matplotlib.transforms import Bbox, TransformedBbox

fig = plt.figure(figsize=(12, 12))
ax = plt.axes()
finalDIM["i_ifg_VV_08Apr2016_20Apr2016"].plot(ax=ax, robust=True)

plt.xlim(500, 2000)
plt.ylim(1000, 3000)
plt.show()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker-geospatial|sentinel1-insar-snap|sentinel1_insar_kumamoto.ipynb)
